In [1]:
# Set the new configuration
conf = SparkConf().setAll([('spark.executor.memory', '4g'),\
                           ('spark.driver.memory', '8g'),\
                           ('spark.shuffle.service.enabled', True), \
                           ('spark.dynamicAllocation.enabled', True), \
                           #('spark.executor.instances', 50)
                           ('spark.dynamicAllocation.executorIdleTimeout', 600), \
                           ('spark.executor.cores', 1),\
                           ('spark.default.parallelism', 90),\
                           ('spark.executor.memoryOverhead', '4g'),\
                           ('spark.driver.memoryOverhead', '4g'),\
                           ('spark.scheduler.mode', 'FAIR'),\
                           ('spark.kryoserializer.buffer.max', '512m'),\
                           ('spark.app.name','LightCurve Demo - JupyterHub Elephas implementation')])# Show the current options




#                           ('spark.dynamicAllocation.maxExecutors', 90), \


# Stop the old context
sc.stop()

# And restart the context with the new configuration
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import math
import copy
import random

import time

from pyspark.ml.feature import StringIndexer, StandardScaler,VectorAssembler,OneHotEncoder
from pyspark.ml import Pipeline

from pyspark.sql.functions import udf, col, array, lit
from pyspark.ml.linalg import Vectors, VectorUDT

from pyspark.sql.functions import rand

from pyspark.sql.types import ArrayType, FloatType,IntegerType, DataType, DoubleType

from pyspark.mllib.evaluation import MulticlassMetrics

from keras import optimizers
from keras.models import Sequential, Model, load_model # model and load_model from Plasticc
from keras.layers import Dense, Dropout, Activation, Layer, Lambda
from keras import backend as K

from elephas.ml_model import ElephasEstimator


from keras.layers import *
from keras.optimizers import Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import to_categorical
from keras.utils import np_utils, generic_utils

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

import tensorflow as tf

Using TensorFlow backend.


WARNING


In [3]:
import sys

In [4]:
sqlContext = SQLContext(sc)
sqlContext.sql("use plasticc")

DataFrame[]

In [5]:
iType=IntegerType()
dType=DoubleType()
fType=FloatType()

In [6]:
augment_count = 25
#batch_size = 128
#batch_size2 = 512
batch_size = 1000
batch_size2 = 5000
optimizer = 'nadam'
num_models = 1
use_specz = False
valid_size = 0.1
max_epochs = 50


In [7]:
classes = np.array([6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, 99], dtype='int32')
class_names = ['class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']
class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1, 99: 1}

# LSST passbands (nm)  u    g    r    i    z    y      
passbands = np.array([357, 477, 621, 754, 871, 1004], dtype='float32')

In [8]:
limit = 1000000
sequence_len = 256
num_classes = len(classes)

#### Get the custom loss functions

In [9]:
def get_wtable(df):
    
    all_y = np.array(df.select('target').collect(), dtype = 'int32') 

    y_count = np.unique(all_y, return_counts=True)[1]

    wtable = np.ones(len(classes))

    for i in range(0, y_count.shape[0]):
        wtable[i] = y_count[i] / all_y.shape[0]

    return wtable    

def mywloss(y_true,y_pred):
    yc=tf.clip_by_value(y_pred,1e-15,1-1e-15)
    loss=-(tf.reduce_mean(tf.reduce_mean(y_true*tf.log(yc),axis=0)/wtable))
    return loss
    
    
def multi_weighted_logloss(y_ohe, y_p, wtable):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1-1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set 
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).astype(float)
    nb_pos = wtable

    if nb_pos[-1] == 0:
        nb_pos[-1] = 1

    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos    
    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss / y_ohe.shape[0]

#### References

https://www.tensorflow.org/api_docs/python/tf/keras/backend/permute_dimensions

#### UDF Functions for vector creation - definitions

In [10]:
def pad_array(x, sequence_len=sequence_len):
    x = np.pad(x, (sequence_len,0), 'constant', constant_values=(0))
    x= x[len(x)-sequence_len:len(x)]
    return x

def fwd_intervals(x):
    x=np.ediff1d(x, to_begin = [0])
    return x

def bwd_intervals(x):
    x=np.ediff1d(x, to_end = [0])
    return x

#### Issues returning numpy arrays to pyspark UDFs.
You'll get pickel errors because the funtion returns NumPy types which are not compatible with DataFrame API. You have to cast the function return back to a list, and then caset to Spark comparible data types.

https://stackoverflow.com/questions/44965762/is-it-possible-to-store-a-numpy-array-in-a-spark-dataframe-column

#### UDF Declarations for vector manipulation

Note that we have included the UDF toDenseUDF - this is necessary because the Spark ML class VectorAssembler will cast the assembled vector to a sparse vector if there are a large number of zeros in the vector. This not usually a problem, but for this problem we do need to pass in a static dense vector into the Keras model in order to properly extract the features.

In [11]:
target_categorical = udf(
    lambda arr:
        [int(i+1 == arr) for i in range(num_classes)], 
        ArrayType(iType)       
)

get_padded_float_vectors = udf(
    lambda arr: pad_array(arr).tolist(), 
    ArrayType(fType)
)

get_padded_int_vectors = udf(
    lambda arr: pad_array(arr).tolist(), 
    ArrayType(iType)
)

toDenseUdf = udf(
    lambda arr: Vectors.dense(arr.toArray()), 
    VectorUDT()
)

fwd_udf = udf(
    lambda arr: fwd_intervals(arr).tolist(), 
    ArrayType(fType)
)

bwd_udf = udf(
    lambda arr: bwd_intervals(arr).tolist(), 
    ArrayType(fType)
)

to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

#### Create the dataframe

In [12]:
trainingVectorsDF=sqlContext.sql("""
select object_id, target,meta, band, mjd, flux, flux_err, detected, fwd_int, bwd_int, 
source_wavelength, received_wavelength
from elephas_training_set""")

In [13]:
wtable=get_wtable(trainingVectorsDF)

### Set up the training set dataframe with the feature vectors

This is how we convert the target to a categegorical, looks like ElephaseSTIMATOE doesn't want it

                             to_vector(target_categorical("target")).alias("targetV"),


In [14]:
trainingVectorsDF = trainingVectorsDF.select(\
                       "object_id",target_categorical("target").alias("target"),
                             "meta",                           
                             get_padded_int_vectors("band").alias("band"),
                             get_padded_float_vectors("mjd").alias("mjd"),
                             get_padded_float_vectors("flux").alias("flux"),
                             get_padded_float_vectors("flux_err").alias("flux_err"),
                             get_padded_int_vectors("detected").alias("detected"),
                             fwd_udf(get_padded_float_vectors("fwd_int")).alias("fwd_int"),
                             bwd_udf(get_padded_float_vectors("bwd_int")).alias("bwd_int"),
                             get_padded_float_vectors("source_wavelength").alias("source_wavelength"),
                             get_padded_float_vectors("received_wavelength").alias("received_wavelength")
                            )

In [15]:
def get_keras_data(sc,trainingVectorsDF, sequence_len=sequence_len, num_classes=num_classes):

    r=trainingVectorsDF.count()
    
    idArr=np.array(trainingVectorsDF.select('object_id').collect(), dtype='int32').reshape(r,)
    #idArr.reshape(r,)
    meta_len=10
    
    metaArr=np.array(trainingVectorsDF.select('meta').collect(), dtype='float32').reshape(r,meta_len)
    bandArr= np.array(trainingVectorsDF.select('band').collect() , dtype='int32').reshape(r,sequence_len)

    histArray=np.zeros((r,sequence_len,8), dtype='float32') 
    # this will work brilliantly as get_keras_data sets three columns to zeros anyway
    
    #mjd=np.array(trainingVectorsDF.select('mjd').collect(), dtype='float32').reshape(r,sequence_len)
    flux=np.array(trainingVectorsDF.select('flux').collect(), dtype='float32').reshape(r,sequence_len)
    flux_err=np.array(trainingVectorsDF.select('flux_err').collect(), dtype='float32').reshape(r,sequence_len)
    #detect=np.array(trainingVectorsDF.select('detected').collect(), dtype='float32').reshape(r,sequence_len)
    fwd_int=np.array(trainingVectorsDF.select('fwd_int').collect(), dtype='float32').reshape(r,sequence_len)
    bwd_int=np.array(trainingVectorsDF.select('bwd_int').collect(), dtype='float32').reshape(r,sequence_len)
    source_wavelength=np.array(trainingVectorsDF.select('source_wavelength').collect(), dtype='float32').reshape(r,sequence_len)
    #received_wavelength=np.array(trainingVectorsDF.select('received_wavelength').collect(), dtype='float32').reshape(r,sequence_len)
    
    #as per the baseline program, we remove the abs time, detected and receoved_wavelength data

    #histArray[:,:,0]=mjd
    histArray[:,:,1]=flux
    histArray[:,:,2]=flux_err
    #histArray[:,:,3]=detect
    histArray[:,:,4]=fwd_int
    histArray[:,:,5]=bwd_int
    histArray[:,:,6]=source_wavelength
    #histArray[:,:,7]=received_wavelength

    # Create the final vector dictionary
    X = {
            'id': idArr,
            'meta': metaArr,
            'band': bandArr,
            'hist': histArray
        }
    # and the encoded target vector
    Y = np.array(trainingVectorsDF.select('target').collect(), dtype='int32').reshape(r, num_classes)  
    
    return X, Y

In [16]:
X, Y = get_keras_data(sc,trainingVectorsDF, sequence_len, num_classes)

In [18]:
X

{'id': array([     3910,      4173,     15718, ...,  98720151, 115870585,
        119178558], dtype=int32),
 'meta': array([[0.       , 0.       , 0.       , ..., 0.009    , 1.       ,
         0.7357895],
        [0.       , 0.       , 0.       , ..., 0.019    , 1.       ,
         0.8051275],
        [0.       , 0.       , 0.       , ..., 0.008    , 1.       ,
         0.6469907],
        ...,
        [0.       , 0.       , 0.       , ..., 0.021    , 1.       ,
         0.7495351],
        [0.       , 0.       , 0.       , ..., 0.046    , 0.       ,
         0.8119878],
        [0.       , 0.       , 0.       , ..., 0.107    , 1.       ,
         0.8051755]], dtype=float32),
 'band': array([[0, 1, 0, ..., 2, 0, 3],
        [0, 0, 2, ..., 5, 4, 1],
        [3, 3, 0, ..., 2, 0, 0],
        ...,
        [0, 0, 0, ..., 4, 5, 5],
        [0, 0, 0, ..., 1, 1, 5],
        [0, 0, 0, ..., 4, 4, 0]], dtype=int32),
 'hist': array([[[ 0.00000000e+00, -8.94139241e-03,  1.49680069e-02, ...,
      

In [ ]:
trainingVectorsDF.printSchema()

In [ ]:
r=trainingVectorsDF.count()

In [ ]:
metaArr=np.array(trainingVectorsDF.select('meta').collect(), dtype='float32').reshape(r,10)

In [ ]:
bandArr= np.array(trainingVectorsDF.select('band').collect() , dtype='int32').reshape(r,256)

In [ ]:
histArray=np.zeros((r,sequence_len,8), dtype='float32') 

In [ ]:
flux=np.array(trainingVectorsDF.select('flux').collect(), dtype='float32').reshape(r,sequence_len)
flux_err=np.array(trainingVectorsDF.select('flux_err').collect(), dtype='float32').reshape(r,sequence_len)
fwd_int=np.array(trainingVectorsDF.select('fwd_int').collect(), dtype='float32').reshape(r,sequence_len)
bwd_int=np.array(trainingVectorsDF.select('bwd_int').collect(), dtype='float32').reshape(r,sequence_len)
source_wavelength=np.array(trainingVectorsDF.select('source_wavelength').collect(), dtype='float32').reshape(r,sequence_len)


In [ ]:
#histArray[:,:,0]=mjd
histArray[:,:,1]=flux
histArray[:,:,2]=flux_err
#histArray[:,:,3]=detect
histArray[:,:,4]=fwd_int
histArray[:,:,5]=bwd_int
histArray[:,:,6]=source_wavelength
#histArray[:,:,7]=received_wavelength

# Create the final vector dictionary
X = {
        'meta': metaArr,
        'band': bandArr,
        'hist': histArray
    }

In [ ]:
X

In [ ]:
augVec=sqlContext.sql("select * from training_set_augmented_vectors")

In [ ]:
augVec.printSchema()

In [ ]:
r=trainingVectorsDF.count()

In [ ]:
bandArr= np.array(augVec.select('band').collect()) #, dtype='int32') #.reshape(r,sequence_len)

In [ ]:
bandArr.shape

In [ ]:
bandArr.reshape(196200, 256)

In [ ]:
X, Y = get_keras_data(sc,trainingVectorsDF)

In [ ]:
idArr=np.array(vectors_df.select('object_id').collect(), dtype='int32')
idArr.reshape(r,)

In [ ]:
meta_len=10
start=time.time()
metaArr=np.array(trainingVectorsDF.select('meta').collect(), dtype='float32').reshape(r,meta_len)
bandArr= np.array(trainingVectorsDF.select('band').collect() , dtype='int32').reshape(r,sequence_len)

histArray=np.zeros((r,sequence_len,8), dtype='float32') 

mjd=np.array(trainingVectorsDF.select('mjd').collect(), dtype='float32').reshape(r,sequence_len)
flux=np.array(trainingVectorsDF.select('flux').collect(), dtype='float32').reshape(r,sequence_len)
flux_err=np.array(trainingVectorsDF.select('flux_err').collect(), dtype='float32').reshape(r,sequence_len)
detect=np.array(trainingVectorsDF.select('detect').collect(), dtype='float32').reshape(r,sequence_len)
fwd_int=np.array(trainingVectorsDF.select('fwd_int').collect(), dtype='float32').reshape(r,sequence_len)
bwd_int=np.array(trainingVectorsDF.select('bwd_int').collect(), dtype='float32').reshape(r,sequence_len)
source_wavelength=np.array(trainingVectorsDF.select('source_wavelength').collect(), dtype='float32').reshape(r,sequence_len)
received_wavelength=np.array(trainingVectorsDF.select('received_wavelength').collect(), dtype='float32').reshape(r,sequence_len)

end=time.time()


In [ ]:
print(end-start)


In [ ]:
histArray[:,:,0]=mjd
histArray[:,:,1]=flux
histArray[:,:,2]=flux_err
histArray[:,:,3]=detect
histArray[:,:,4]=fwd_int
histArray[:,:,5]=bwd_int
histArray[:,:,6]=source_wavelength
histArray[:,:,7]=received_wavelength

# Create the final vector dictionary
X = {
        'id': idArr,
        'meta': metaArr,
        'band': bandArr,
        'hist': histArray
    }
# and the encoded target vector
Y = np.array(vectors_df.select('target').collect(), dtype='int32')

#return X, Y

In [ ]:
df_pdf=trainingVectorsDF.toPandas()


In [ ]:
target=df_pdf["target"].to_numpy()

In [ ]:
trainingVectorsDF = trainingVectorsDF.select(\
                       "object_id","target", #target_categorical("target").alias("target"),
                             to_vector("meta").alias("meta"),                           
                             to_vector(get_padded_int_vectors("band")).alias("band"),
                             to_vector(get_padded_float_vectors("mjd")).alias("mjd"),
                             to_vector(get_padded_float_vectors("flux")).alias("flux"),
                             to_vector(get_padded_float_vectors("flux_err")).alias("flux_err"),
                             to_vector(get_padded_int_vectors("detected")).alias("detect"),
                             to_vector(fwd_udf(get_padded_float_vectors("fwd_int"))).alias("fwd_int"),
                             to_vector(bwd_udf(get_padded_float_vectors("bwd_int"))).alias("bwd_int"),
                             to_vector(get_padded_float_vectors("source_wavelength")).alias("source_wavelength"),
                             to_vector(get_padded_float_vectors("received_wavelength")).alias("received_wavelength")
                            )

In [ ]:
trainingVectorsDF.printSchema()

In [ ]:
ignore = ['object_id', 'target']

assembler = VectorAssembler(
    inputCols=[x for x in trainingVectorsDF.columns if x not in ignore],
    outputCol='features')

trainingVectorsDF=assembler.transform(trainingVectorsDF)

In [ ]:
trainingVectorsDF.printSchema()

In [ ]:
weights = [.6, .3, .1]
seed = 42 # seed=0L  validation_df, 
train_df, test_df, validation_df = trainingVectorsDF.select("object_id","target", toDenseUdf("features").alias("features")).randomSplit(weights, seed)

In [ ]:
r=train_df.count()

In [ ]:
features=np.array(train_df.select("features").collect())

In [ ]:
trainingVectorsDF.take(2)

### The training, test and validation splits

Note that this is where we apply the toDenseUDF function to ensure that the features vector is a dense vector

weights = [.7, .3]
seed = 42 # seed=0L  validation_df, 
train_df, test_df = trainingVectorsDF.select( toDenseUdf("scaled_features").alias("features"), "target").randomSplit(weights, seed)

In [ ]:
weights = [.6, .3, .1]
seed = 42 # seed=0L  validation_df, 
train_df, test_df, validation_df = trainingVectorsDF.select( toDenseUdf("features").alias("features"), "target").randomSplit(weights, seed)

In [ ]:
train_df.printSchema()

In [ ]:
train_df=train_df.repartition(800)

In [ ]:
nb_classes = len(classes)
#input_dim = len(train_df.select("features").first()[0])
input_dim = train_df.select("features").first()[0].shape

print(f"We have {num_classes} classes and {input_dim[0]} features")


## Preprocessing: Defining Transformers

Up until now, we basically just read in raw data. Luckily, ```Spark ML``` has quite a few preprocessing features available, so the only thing we will ever have to do is define transformations of data frames.

To proceed, we will first transform category strings to double values. This is done by a so called ```StringIndexer```. Note that we carry out the actual transformation here already, but that is just for demonstration purposes. All we really need is too define ```string_indexer``` to put it into a pipeline later on.

#### get and compile the model 

In [ ]:
model = get_model(train_df, input_dim)
print("hi there")

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizer, loss=mywloss, metrics=['accuracy'])
print("hi there")

## Distributed Elephas model

To lift the above Keras ```model``` to Spark, we define an ```Estimator``` on top of it. An ```Estimator``` is Spark's incarnation of a model that still has to be trained. It essentially only comes with only a single (required) method, namely ```fit```. Once we call ```fit``` on a data frame, we get back a ```Model```, which is a trained model with a ```transform``` method to predict labels.

We do this by initializing an ```ElephasEstimator``` and setting a few properties. As by now our input data frame will have many columns, we have to tell the model where to find features and labels by column name. Then we provide serialized versions of our Keras model. We can not plug in keras models into the ```Estimator``` directly, as Spark will have to serialize them anyway for communication with workers, so it's better to provide the serialization ourselves. In fact, while pyspark knows how to serialize ```model```, it is extremely inefficient and can break if models become too large. Spark ML is especially picky (and rightly so) about parameters and more or less prohibits you from providing non-atomic types and arrays of the latter. Most of the remaining parameters are optional and rather self explainatory. Plus, many of them you know if you have ever run a keras model before. We just include them here to show the full set of training configuration.

In [ ]:
#adam=optimizers.nadam(lr=0.01)
adam=optimizers.nadam(lr=0.01)
opt_conf = optimizers.serialize(adam)


In [ ]:
print(max_epochs)

In [ ]:
#optimizer = 'nadam'
adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)

# Initialize SparkML Estimator and set all relevant properties
estimator = ElephasEstimator()
estimator.setFeaturesCol("features")             # These two come directly from pyspark,
estimator.setLabelCol("target")                 # hence the camel case. Sorry :)
estimator.set_keras_model_config(model.to_yaml())       # Provide serialized Keras model
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(80)  # We just use one worker here. Feel free to adapt it.
estimator.set_epochs(max_epochs) 
estimator.set_batch_size(batch_size) # was 128
estimator.set_verbosity(2) # was 1
estimator.set_validation_split(0.15)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous") # Was synchronous
estimator.set_loss(mywloss) # was("categorical_crossentropy")
estimator.set_metrics(['accuracy']) ##(['acc'])

## And now we set up the pipeline.

Looks very similar to SparkFlow, n'est ce pas? This could be an interesting comparison!

Defining pipelines is really as easy as listing pipeline stages. We can provide any configuration of Transformers and Estimators really, but here we simply take the three components defined earlier. Note that string_indexer and scaler and interchangable, while estimator somewhat obviously has to come last in the pipeline.

In [ ]:
pipeline = Pipeline(stages=[estimator])

## And train the model

Note that at this stage, the only method we can call is ''fit''

In [ ]:
import time

start=time.time()
fitted_pipeline = pipeline.fit(train_df) # Fit model to data
elapsedTime=time.time()-start
print(f"Model trained in {elapsedTime} seconds")

#### Save the model

#### Now we run the transform so we can get the predictions

However, this will require modifications to the class file for ElephasEstimator and ElephasTransformer located here

 ~/.virtualenvs/Elephas/lib/python3.6/site-packages/elephas/ml_model.py
 
because the \_transform method as written runs the predict process using the model.predict_classes method. A keras Sequential() model class has this method, but the Model() class does not - you have to use the 'predict' method.

What this entails is that you have to rewrite that method and overload it to incorporate the predict method if you're training on a keras Model class instead of a Sequential model.

The overloaded method is described here - 

https://github.com/maxpumperla/elephas/issues/111

and we will include a copy of the modified class statement in th github



In [ ]:
validation_df=validation_df.repartition(400)

In [ ]:
pred = fitted_pipeline.stages[-1]._transform(validation_df, useModel=True)

In [ ]:
pred.printSchema()

In [ ]:
pnl=pred.select("target","prediction")
pnl.show(100)

In [ ]:
# Looks like prediction_and_label is a dataframe, not an RDD, so we need to cast it to an RDD in order to use .map
prediction_and_label = pnl.rdd.map(lambda row: (row.target, row.prediction))
metrics = MulticlassMetrics(prediction_and_label)
print(metrics.precision())

In [ ]:
x1=np.array([0.1118802 , 0.07325512, 0.01614694, 0.02659923, 0.07440449,
        0.08224725, 0.06605115, 0.02206622, 0.0542385 , 0.04232709,
        0.05887228, 0.0560729 , 0.12828934, 0.10508711, 0.08246218], dtype='float32')
x2=np.array([0.11386207, 0.09637289, 0.01490068, 0.02821412, 0.06656378,
        0.080683  , 0.07982644, 0.01722281, 0.04762489, 0.03941712,
        0.07437494, 0.06128084, 0.08523843, 0.09439979, 0.10001823], dtype='float32')

In [ ]:
np.argmax(x1)

In [ ]:
np.argmax(x2)

In [ ]:
20393/3600